Este jupyter notebook nos ayudará a entender qué es hacer web scraping en una página web desde un punto de vista práctico. El contenido de este jupyter notebook ha sido adaptado del capítulo 8 del libro de texto ***Getting Started with Beautiful Soup*** de Vineeth G. Nair (2014).

# Primer Paso: 
## Definir cuál es la página web desde donde vamos a extraer los datos

Aquí vamos a utilizar la página web [packtpub.com](https://www.packtpub.com/). Sobre esta página el scraper que construiremos nos permitirá extraer los datos del precio de una lista de libros vendidos en esa página web y buscar en otras páginas como [amazon.com](www.amazon.com) o [barnes and nobles](www.barnesandnobles.com) empleando el ISBN de cada libro (el ISBN de un libro es el equivalente a la cédula de identidad de una persona. Ayuda a identificar inequívocamente al libro).

# Segundo Paso:

## Explorar el contenido de la página web 

En este caso, solo vamos a concentrarnos en el título de cada libro, su precio de venta y su ISBN. Para explorar el contenido, primero ingresamos a la página web con un navegador y miramos su contenido como lo hacemos usualmente. Luego, 